In [69]:
import numpy as np
import torch
from torch.utils.data import TensorDataset
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
import torchmetrics

In [6]:
train_data = pd.read_csv("data/electricity_train.csv")
test_data = pd.read_csv("data/electricity_test.csv")

In [7]:
def create_sequences(df, seq_length):
    xs, ys = [], []
    # Iterate over data indices
    for i in range(len(df) - seq_length):
      	# Define inputs
        x = df.iloc[i:(i+seq_length), 1]
        # Define target
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [8]:
# Use create_sequences to create inputs and targets
X_train, y_train = create_sequences(train_data, 96)
print(X_train.shape, y_train.shape)

(105119, 96) (105119,)


In [65]:
X_test, y_test = create_sequences(test_data, 96)

In [9]:
# Create TensorDataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
)
print(len(dataset_train))

105119


In [66]:
dataset_test = TensorDataset(
    torch.from_numpy(X_test).float(),
    torch.from_numpy(y_test).float(),
)
print(len(dataset_test))

34944


In [32]:
dataloader_train = DataLoader(
    dataset_train,
    shuffle=True,
    batch_size=16
)

In [67]:
dataloader_test = DataLoader(
    dataset_test,
    shuffle=True,
    batch_size=16
)

In [58]:
for seq, label in dataloader_train:
    print(seq.shape)
    print(label.shape)

torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([16, 96])
torch.Size([16])
torch.Size([

## RNN

In [49]:
#Building a forecasting RNN

class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size = 1,
            hidden_size = 32,
            num_layers = 2,
            batch_first = True,
        )
        self.fc = nn.Linear(32, 1)
    
    def forward(self, x):
        # Initialize first hidden state with zeros
        h0 = torch.zeros(2, x.size(0), 32)
        # Pass x and h0 through recurrent layer
        out, _ = self.rnn(x, h0)  
        # Pass recurrent layer's last output through linear layer
        out = self.fc(out[:, -1, :])
        return out



## LSTM

In [60]:
#Building LSTM

class LSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Define lstm layer
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        # Initialize long-term memory
        c0 = torch.zeros(2, x.size(0), 32)
        # Pass all inputs to lstm layer
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

## GRU

In [71]:
#Building GRU

class GRU(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Define RNN layer
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.gru(x, h0)  
        out = self.fc(out[:, -1, :])
        return out

### LSTM training loop

In [ ]:
lstm = LSTM(1)
# Set up MSE loss
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(3):
    for seqs, labels in dataloader_train:
        # Reshape model inputs
        seqs = seqs.view(seqs.size(0), 96, 1)
        # Get model outputs
        outputs = lstm(seqs)
        # Compute loss
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [70]:
# Define MSE metric
mse = torchmetrics.MeanSquaredError()

lstm.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(seqs.size(0), 96, 1)
        # Pass seqs to net and squeeze the result
        outputs = lstm(seqs).squeeze()
        mse(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")

Test MSE: 0.6516733765602112


### GRU training loop

In [72]:
gru = GRU(1)
# Set up MSE loss
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(3):
    for seqs, labels in dataloader_train:
        # Reshape model inputs
        seqs = seqs.view(seqs.size(0), 96, 1)
        # Get model outputs
        outputs = gru(seqs)
        # Compute loss
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

/opt/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.5318925976753235
Epoch 2, Loss: 0.9346930384635925
Epoch 3, Loss: 0.79939204454422


In [73]:
# Define MSE metric
mse = torchmetrics.MeanSquaredError()

gru.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(seqs.size(0), 96, 1)
        # Pass seqs to net and squeeze the result
        outputs = gru(seqs).squeeze()
        mse(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")

Test MSE: 0.6435002684593201
